# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Introduction: Business Problem <a name="introduction"></a>

Hue and Da Nag are two cities in central Vietnam. They are both on the coast of Vietnam, but Da Nang is more renowned for beaches than Hue. 
Hue on the other hand is known for being the former center of power in Vietnam and was the sett of Royalty until 1947.  
Both a major tourist attractions, or at least were until Covid19. 
Originally I wanted to compare the influence on the cities of the Covid19 lockdown in the fall of 2020 because Da Nang was in lockdown and Hue wasn't. 
As it is not possible to gather data from Foursquare based on date I will compare the two cities as they stand today.
In particular I will look at the tips and see if there is a correlation between the length of the tip and the rating given.


## Data<a name="data"></a>

In [1]:
import pandas as pd
import numpy as np
!pip install geocoder
import geocoder

     |████████████████████████████████| 98 kB 6.7 MB/s  eta 0:00:01


In [2]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 94 kB 3.9 MB/s  eta 0:00:01
Libraries imported.


Find the latitude and longitude of Da Nang, Vietnam

In [3]:
address = 'Da Nang, VN'

geolocator = Nominatim(user_agent="da_nang_explorer")
location_danang = geolocator.geocode(address)
latitude_danang = location_danang.latitude
longitude_danang = location_danang.longitude
print('The geograpical coordinate of Da Nang is {}, {}.'.format(latitude_danang, longitude_danang))

The geograpical coordinate of Da Nang is 16.068, 108.212.


Find the latitude and longitude of Hue, Vietnam

In [4]:
address = 'Hue, VN'

geolocator2 = Nominatim(user_agent="hue_explorer")
location = geolocator2.geocode(address)
latitude_hue = location.latitude
longitude_hue = location.longitude
print('The geograpical coordinate of Hue is {}, {}.'.format(latitude_hue, longitude_hue))

The geograpical coordinate of Hue is 16.4638013, 107.5821911.


Lets look at the locations of Hue and Da Nang on a map

In [5]:
# create map of Da Nang and Hue using latitude and longitude values
map_cities = folium.Map(location=[((latitude_danang + latitude_hue)/2), ((longitude_danang + longitude_hue)/2)], zoom_start=10)

folium.Marker(
    location=[latitude_danang, longitude_danang],
    popup="Da Nang",
    icon=folium.Icon(icon="green"),
).add_to(map_cities)

folium.Marker(
    location=[latitude_hue, longitude_hue],
    popup="Hue",
    icon=folium.Icon(icon="green"),
).add_to(map_cities)

    
map_cities

FourSquare details

In [39]:
# The code was removed by Watson Studio for sharing.

Get some information from FourSquare about Da Nang

In [40]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_danang, 
    longitude_danang, 
    radius, 
    LIMIT)

In [41]:
results_danang = requests.get(url).json()
results_danang

{'meta': {'code': 200, 'requestId': '601a216ff5e9ca325a155922'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Da Nang',
  'headerFullLocation': 'Da Nang',
  'headerLocationGranularity': 'city',
  'totalResults': 130,
  'suggestedBounds': {'ne': {'lat': 16.104000036000038,
    'lng': 108.249393644179},
   'sw': {'lat': 16.031999963999965, 'lng': 108.17460635582101}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '582b1746641e5c276f013971',
       'name': "Pizza 4P's",
       'contact': {},
       'location': {'address': '8 Hoàng Văn Thụ',
        'lat': 16.062674975021636,
        'lng': 108.2228449147913,
        'labeledLatLngs': [{'label': 'display',
          'lat': 16.06267

In [42]:
items_danang = results_danang['response']['groups'][0]['items']
items_danang[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '582b1746641e5c276f013971',
  'name': "Pizza 4P's",
  'contact': {},
  'location': {'address': '8 Hoàng Văn Thụ',
   'lat': 16.062674975021636,
   'lng': 108.2228449147913,
   'labeledLatLngs': [{'label': 'display',
     'lat': 16.062674975021636,
     'lng': 108.2228449147913}],
   'distance': 1302,
   'cc': 'VN',
   'city': 'Đà Nẵng',
   'state': 'Thành Phố Đà Nẵng',
   'country': 'Việt Nam',
   'formattedAddress': ['8 Hoàng Văn Thụ',
    'Đà Nẵng',
    'Thành Phố Đà Nẵng',
    'Việt Nam']},
  'categories': [{'id': '4bf58dd8d48988d1ca941735',
    'name': 'Pizza Place',
    'pluralName': 'Pizza Places',
    'shortName': 'Pizza',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
     'suffix': '.png'},
    'primary': True}],
  'verified': False,
  'stats': {'tipCount': 0,
   'usersCount': 0,
   'check

In [43]:
#fetching omly relevant data from categories
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#flatten JSON, Normalize JSON to Dataframe
dataframe = json_normalize(items_danang) 

# filter columns,consider only required columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
nearby_danang = dataframe.loc[:, filtered_columns]

# filter the category for each row
nearby_danang['venue.categories'] = nearby_danang.apply(get_category_type, axis=1)

# clean columns
nearby_danang.columns = [col.split('.')[-1] for col in nearby_danang.columns]

#replce NaN values with Not found in address
nearby_danang['address'] = nearby_danang['address'].fillna("Not found")
nearby_danang.head(10)   

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,neighborhood,id
0,Pizza 4P's,Pizza Place,8 Hoàng Văn Thụ,16.062675,108.222845,"[{'label': 'display', 'lat': 16.06267497502163...",1302,VN,Đà Nẵng,Thành Phố Đà Nẵng,Việt Nam,"[8 Hoàng Văn Thụ, Đà Nẵng, Thành Phố Đà Nẵng, ...",NaN,NaN,NaN,582b1746641e5c276f013971
1,cộng cà phê,Coffee Shop,39,16.068063,108.223512,"[{'label': 'display', 'lat': 16.06806287945548...",1231,VN,Đà Nẵng,Thành Phố Đà Nẵng,Việt Nam,"[39 (Nguyễn Thái Học), Đà Nẵng, Thành Phố Đà N...",Nguyễn Thái Học,551105,NaN,5a26a41a31ac6c676705e94c
2,Sanouva Danang Hotel (Khách Sạn Sanouva Đà Nẵng),Hotel,"68 Phan Chau Trinh, Hai Chau Dist., Da Nang City",16.066620,108.220031,"[{'label': 'display', 'lat': 16.06661966448064...",872,VN,Đà Nẵng,Thành Phố Đà Nẵng,Việt Nam,"[68 Phan Chau Trinh, Hai Chau Dist., Da Nang C...",68 Phan Chau Trinh,550000,NaN,546b5fc2498e85d6109593cc
3,Thìa Gỗ Danang-style restaurant (Quán ăn ngon ...,Vietnamese Restaurant,37 Ba Đinh,16.076115,108.218690,"[{'label': 'display', 'lat': 16.07611473938317...",1152,VN,Đà Nẵng,Thành Phố Đà Nẵng,Việt Nam,"[37 Ba Đinh, Đà Nẵng, Thành Phố Đà Nẵng, Việt ...",NaN,551300,NaN,5aabf13ed552c7495c4b16cc
4,Bánh Bèo Tâm,Snack Place,291 Nguyễn Chí Thanh,16.064597,108.220541,"[{'label': 'display', 'lat': 16.06459685641845...",989,VN,NaN,Da Nang,Việt Nam,"[291 Nguyễn Chí Thanh, Da Nang, Việt Nam]",NaN,NaN,NaN,4e13f98518388432510378f9
5,Mì Quảng Bà Mua,Vietnamese Restaurant,21 Trần Bình Trọng,16.065974,108.219153,"[{'label': 'display', 'lat': 16.06597405743636...",797,VN,Đà Nẵng,Thành Phố Đà Nẵng,Việt Nam,"[21 Trần Bình Trọng, Đà Nẵng, Thành Phố Đà Nẵn...",NaN,NaN,NaN,4f842b4ce4b05b009467b674
6,Nướng Đà Thành,BBQ Joint,Lô 1-2 B2.4 Nguyễn Văn Linh,16.060689,108.218386,"[{'label': 'display', 'lat': 16.06068929153090...",1062,VN,Đà Nẵng,Thành Phố Đà Nẵng,Việt Nam,[Lô 1-2 B2.4 Nguyễn Văn Linh (Huỳnh Thúc Kháng...,Huỳnh Thúc Kháng,NaN,NaN,5405d20d498eae2180a53eab
7,Bánh Canh Nga 78 Nguyễn Chí Thanh,Soup Place,78 Nguyễn Chí Thanh,16.072862,108.220726,"[{'label': 'display', 'lat': 16.07286246561564...",1079,VN,Đà Nẵng,Thành Phố Đà Nẵng,Việt Nam,"[78 Nguyễn Chí Thanh, Đà Nẵng, Thành Phố Đà Nẵ...",NaN,NaN,NaN,516a82b6e4b0d46e577ac8b8
8,Tre Việt restaurant,Vietnamese Restaurant,180 Bach Dang,16.066288,108.224493,"[{'label': 'display', 'lat': 16.06628814061497...",1349,VN,Đà Nẵng,Thành Phố Đà Nẵng,Việt Nam,"[180 Bach Dang, Đà Nẵng, Thành Phố Đà Nẵng, Vi...",NaN,NaN,NaN,583a7e2581907921f5404ea2
9,Cafe 379,Coffee Shop,37-39 Pasteur,16.069699,108.218045,"[{'label': 'display', 'lat': 16.06969870761500...",673,VN,Đà Nẵng,Thành Phố Đà Nẵng,Việt Nam,"[37-39 Pasteur, Đà Nẵng, Thành Phố Đà Nẵng, Vi...",NaN,NaN,NaN,4e83d431be7b87d6abc6d521


In [44]:
# creating map zoom at present Location
map_danang = folium.Map(location=[latitude_danang,longitude_danang],zoom_start=14)

# adding the current location to map
folium.CircleMarker(location=[latitude_danang,longitude_danang],radius=5,color='red',fill=True,fill_color='red').add_to(map_danang)
folium.Marker(location=[latitude_danang,longitude_danang],popup="<strong> Current Location</strong>",icon = folium.Icon(color='red')).add_to(map_danang)

# drawing radius of search
folium.Circle(location=[latitude_danang,longitude_danang],radius=radius,color='blue',opacity=0.5,fill=True,fill_color='blue').add_to(map_danang)

# adding all the popular places around
for lat,lon,name,cate,addr,dist in zip(nearby_danang.lat,nearby_danang.lng,nearby_danang.name,nearby_danang.categories,nearby_danang.address,nearby_danang.distance):
  folium.CircleMarker(location=[lat,lon],radius=5,color="red",fill=True,fill_color='red').add_to(map_danang)
  folium.Marker(location=[lat,lon],
              popup= folium.Popup(('<strong><font color ="red">Name :     </font>'+name+'</strong><br>'+
                                   '<strong><font color ="red">Categorie :    </font>'+cate+'</strong><br>'+
                                   '<strong><font color ="red">Address :    </font>'+addr+'</strong><br>'+
                                   '<strong><font color ="red">Distance :    </font>'+str(dist)+' mts</strong>'
                                   ),max_width=250)).add_to(map_danang)

map_danang

In [46]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_hue, 
    longitude_hue, 
    radius, 
    LIMIT)

In [47]:
results_hue = requests.get(url).json()
results_hue

{'meta': {'code': 200, 'requestId': '601a21a266f6fc1eaf38b4a6'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Huế',
  'headerFullLocation': 'Huế',
  'headerLocationGranularity': 'city',
  'totalResults': 62,
  'suggestedBounds': {'ne': {'lat': 16.499801336000036,
    'lng': 107.619660190731},
   'sw': {'lat': 16.427801263999964, 'lng': 107.54472200926901}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd1ab4a5e0cce720acfa184',
       'name': 'La Residence Hue Hotel & Spa',
       'contact': {},
       'location': {'address': '5 Le Loi Street',
        'lat': 16.459127862940925,
        'lng': 107.58007233528453,
        'labeledLatLngs': [{'label': 'display',
          'lat'

In [48]:
items_hue = results_hue['response']['groups'][0]['items']
items_hue[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4bd1ab4a5e0cce720acfa184',
  'name': 'La Residence Hue Hotel & Spa',
  'contact': {},
  'location': {'address': '5 Le Loi Street',
   'lat': 16.459127862940925,
   'lng': 107.58007233528453,
   'labeledLatLngs': [{'label': 'display',
     'lat': 16.459127862940925,
     'lng': 107.58007233528453}],
   'distance': 567,
   'postalCode': '530000',
   'cc': 'VN',
   'city': 'Huế',
   'state': 'Tỉnh Thừa Thiên-Huế',
   'country': 'Việt Nam',
   'formattedAddress': ['5 Le Loi Street',
    'Huế',
    'Tỉnh Thừa Thiên-Huế',
    'Việt Nam']},
  'categories': [{'id': '4bf58dd8d48988d132951735',
    'name': 'Hotel Pool',
    'pluralName': 'Hotel Pools',
    'shortName': 'Pool',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/pool_',
     'suffix': '.png'},
    'primary': True}],
  'verified': True,
  'stats'

In [49]:
#fetching omly relevant data from categories
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#flatten JSON, Normalize JSON to Dataframe
dataframe = json_normalize(items_hue) 

# filter columns,consider only required columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
nearby_hue = dataframe.loc[:, filtered_columns]

# filter the category for each row
nearby_hue['venue.categories'] = nearby_hue.apply(get_category_type, axis=1)

# clean columns
nearby_hue.columns = [col.split('.')[-1] for col in nearby_hue.columns]

#replce NaN values with Not found in address
nearby_hue['address'] = nearby_hue['address'].fillna("Not found")
nearby_hue.head(10)   

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,La Residence Hue Hotel & Spa,Hotel Pool,5 Le Loi Street,16.459128,107.580072,"[{'label': 'display', 'lat': 16.45912786294092...",567,530000,VN,Huế,Tỉnh Thừa Thiên-Huế,Việt Nam,"[5 Le Loi Street, Huế, Tỉnh Thừa Thiên-Huế, Vi...",NaN,4bd1ab4a5e0cce720acfa184
1,Lạc Thiên,Vietnamese Restaurant,6A Đinh Tiên Hoàng,16.468668,107.585144,"[{'label': 'display', 'lat': 16.468668, 'lng':...",626,NaN,VN,Huế,Tỉnh Thừa Thiên-Huế,Việt Nam,"[6A Đinh Tiên Hoàng (Trần Hưng Đạo), Huế, Tỉnh...",Trần Hưng Đạo,4d3ab780325ff04df0dc1e45
2,Nook Cafe & Bar,Breakfast Spot,7/34 Nguyen Tri Phuong,16.464730,107.593040,"[{'label': 'display', 'lat': 16.46473, 'lng': ...",1162,NaN,VN,Huế,Tỉnh Thừa Thiên-Huế,Việt Nam,"[7/34 Nguyen Tri Phuong, Huế, Tỉnh Thừa Thiên-...",NaN,583848f313bb7732ce36366c
3,Jade Hotel,Hotel,17 Nguyen Thai Hoc,16.468336,107.596721,"[{'label': 'display', 'lat': 16.46833618910892...",1631,NaN,VN,Huế,Tỉnh Thừa Thiên-Huế,Việt Nam,"[17 Nguyen Thai Hoc, Huế, Tỉnh Thừa Thiên-Huế,...",NaN,4ef6e45a722e34060ee19081
4,Nina's Cafe,Vietnamese Restaurant,16/34 Nguyễn Trí Phương,16.464284,107.592762,"[{'label': 'display', 'lat': 16.46428415334502...",1129,NaN,VN,Huế,Tỉnh Thừa Thiên-Huế,Việt Nam,"[16/34 Nguyễn Trí Phương, Huế, Tỉnh Thừa Thiên...",NaN,4c5cdff56ebe2d7faa8bd12e
5,The Scarlett Boutique Hotel,Hotel,Not found,16.467528,107.594623,"[{'label': 'display', 'lat': 16.467528, 'lng':...",1390,NaN,VN,NaN,NaN,Việt Nam,[Việt Nam],NaN,57c5bd65498eb3403f51e7e6
6,Madam Thu Restaurant_Local Specialty Restaurant,Vietnamese Restaurant,45 Vo Thi Sau,16.469847,107.595778,"[{'label': 'display', 'lat': 16.46984667233698...",1598,530000,VN,Huế,Tỉnh Thừa Thiên-Huế,Việt Nam,"[45 Vo Thi Sau (Chu Van An), Huế, Tỉnh Thừa Th...",Chu Van An,58e91776668af70c79eefb81
7,Alba Hotel,Bed & Breakfast,29 Trần Quang Khải,16.465119,107.595372,"[{'label': 'display', 'lat': 16.46511949163949...",1414,530000,VN,Huế,Tỉnh Thừa Thiên-Huế,Việt Nam,"[29 Trần Quang Khải, Huế, Tỉnh Thừa Thiên-Huế,...",NaN,557ba6a6498ed9cb18d574cf
8,Hạnh,Vietnamese Restaurant,11 Phó Đức Chính,16.466283,107.595224,"[{'label': 'display', 'lat': 16.466283, 'lng':...",1418,NaN,VN,Huế,Tỉnh Thừa Thiên-Huế,Việt Nam,"[11 Phó Đức Chính, Huế, Tỉnh Thừa Thiên-Huế, V...",NaN,51d41e52498e96394f5e347d
9,Eldora Hotel,Hotel,60 Ben Nghe Street,16.464897,107.594604,"[{'label': 'display', 'lat': 16.46489686737829...",1330,NaN,VN,Huế,Tỉnh Thừa Thiên-Huế,Việt Nam,"[60 Ben Nghe Street, Huế, Tỉnh Thừa Thiên-Huế,...",NaN,53777885498e4473b30592d9


In [50]:
# creating map zoom at present Location
map_hue = folium.Map(location=[latitude_hue,longitude_hue],zoom_start=14)

# adding the current location to map
folium.CircleMarker(location=[latitude_hue,longitude_hue],radius=5,color='red',fill=True,fill_color='red').add_to(map_hue)
folium.Marker(location=[latitude_hue,longitude_hue],popup="<strong> Current Location</strong>",icon = folium.Icon(color='red')).add_to(map_hue)

# drawing radius of search
folium.Circle(location=[latitude_hue,longitude_hue],radius=radius,color='blue',opacity=0.5,fill=True,fill_color='blue').add_to(map_hue)

# adding all the popular places around
for lat,lon,name,cate,addr,dist in zip(nearby_hue.lat,nearby_hue.lng,nearby_hue.name,nearby_hue.categories,nearby_hue.address,nearby_hue.distance):
  folium.CircleMarker(location=[lat,lon],radius=5,color="red",fill=True,fill_color='red').add_to(map_hue)
  folium.Marker(location=[lat,lon],
              popup= folium.Popup(('<strong><font color ="red">Name :     </font>'+name+'</strong><br>'+
                                   '<strong><font color ="red">Categorie :    </font>'+cate+'</strong><br>'+
                                   '<strong><font color ="red">Address :    </font>'+addr+'</strong><br>'+
                                   '<strong><font color ="red">Distance :    </font>'+str(dist)+' mts</strong>'
                                   ),max_width=250)).add_to(map_hue)

map_hue

In [52]:
# create map of Da Nang and Hue using latitude and longitude values
map_cities = folium.Map(location=[((latitude_danang + latitude_hue)/2), ((longitude_danang + longitude_hue)/2)], zoom_start=10)

# drawing radius of search
folium.Circle(location=[latitude_hue,longitude_hue],radius=radius,color='blue',opacity=0.5,fill=True,fill_color='blue').add_to(map_cities)

# adding all the popular places around in Hue
for lat,lon,name,cate,addr,dist in zip(nearby_hue.lat,nearby_hue.lng,nearby_hue.name,nearby_hue.categories,nearby_hue.address,nearby_hue.distance):
  folium.CircleMarker(location=[lat,lon],radius=5,color="red",fill=True,fill_color='red').add_to(map_hue)
  folium.Marker(location=[lat,lon],
              popup= folium.Popup(('<strong><font color ="red">Name :     </font>'+name+'</strong><br>'+
                                   '<strong><font color ="red">Categorie :    </font>'+cate+'</strong><br>'+
                                   '<strong><font color ="red">Address :    </font>'+addr+'</strong><br>'+
                                   '<strong><font color ="red">Distance :    </font>'+str(dist)+' mts</strong>'
                                   ),max_width=250)).add_to(map_cities)

# drawing radius of search
folium.Circle(location=[latitude_danang,longitude_danang],radius=radius,color='blue',opacity=0.5,fill=True,fill_color='blue').add_to(map_cities)

# adding all the popular places around in Da Nang
for lat,lon,name,cate,addr,dist in zip(nearby_danang.lat,nearby_danang.lng,nearby_danang.name,nearby_danang.categories,nearby_danang.address,nearby_danang.distance):
  folium.CircleMarker(location=[lat,lon],radius=5,color="red",fill=True,fill_color='red').add_to(map_danang)
  folium.Marker(location=[lat,lon],
              popup= folium.Popup(('<strong><font color ="red">Name :     </font>'+name+'</strong><br>'+
                                   '<strong><font color ="red">Categorie :    </font>'+cate+'</strong><br>'+
                                   '<strong><font color ="red">Address :    </font>'+addr+'</strong><br>'+
                                   '<strong><font color ="red">Distance :    </font>'+str(dist)+' mts</strong>'
                                   ),max_width=250)).add_to(map_cities)
    
map_cities

## Methodology<a name="methodology"></a>

## Analysis<a name="analysis"></a>

## Results<a name="results"></a>

## Conclusion<a name="conclusion"></a>